## __Check first before starting__

In [1]:
import os

Working_directory = os.path.normpath("/mnt/mydisk/Continual_Learning_JL/Continual_Learning/")
os.chdir(Working_directory)
print(f"Working directory: {os.getcwd()}")

Working directory: /mnt/mydisk/Continual_Learning_JL/Continual_Learning


## __All imports__

In [2]:
# Operating system and file management
import os
import shutil
import contextlib
import traceback
import gc
import copy
from collections import defaultdict
import subprocess
import time
import re, pickle
import scipy.io
from scipy.io import loadmat
from glob import glob
from math import ceil
from copy import deepcopy

# Jupyter notebook widgets and display
import ipywidgets as widgets
from IPython.display import display

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Plotting and visualization
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory, panhandler

# Machine learning and preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
from ta import trend, momentum, volatility, volume

# Mathematical and scientific computing
import math
from scipy.ndimage import gaussian_filter1d

# Type hinting
from typing import Callable, Tuple

# Deep learning with PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision.models import resnet18
from sklearn.utils.class_weight import compute_class_weight

## __📁 Path Settings and Constants__
This cell defines essential paths and constants for the CPSC2018 ECG dataset processing:
- `BASE_DIR`: Root directory of the project.
- `save_dir`: Path to the preprocessed `.npy` files (one for each continual learning period).
- `ECG_PATH`: Directory containing original `.mat` and `.hea` files.
- `MAX_LEN`: Length of each ECG sample, fixed to 5000 time steps (i.e., 10 seconds at 500Hz).

In [3]:
BASE_DIR = "/mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL"
save_dir = os.path.join(BASE_DIR, "processed")
ECG_PATH = os.path.join(BASE_DIR, "datas")
MAX_LEN = 5000

## __🏷️ Label Mapping and Period Configuration__

This section defines:
- `snomed_map`: Mapping from SNOMED CT codes to readable class names for 9 major ECG conditions.
- `period_label_map`: Incremental learning task structure across four periods.  
  Class `1` is reserved for "OTHER" abnormalities until Period 4 when all 9 classes are explicitly categorized.
- `print_class_distribution()`: Helper function to show class-wise data distribution.


In [4]:
# SNOMED CT to readable names
snomed_map = {
    "426783006": "NSR",    # 正常竇性心律
    "270492004": "I-AVB",  # 一度房室傳導阻滯
    "164889003": "AF",     # 心房纖維顫動
    "164909002": "LBBB",   # 左束支傳導阻滯
    "59118001":  "RBBB",   # 右束支傳導阻滯
    "284470004": "PAC",    # 心房早期搏動
    "164884008": "PVC",    # 室性早期搏動
    "429622005": "STD",    # ST 段壓低
    "164931005": "STE"     # ST 段抬高
}

# Period class mapping (固定 class 1 是「其他異常」直到 P4 移除)
period_label_map = {
    1: {"NSR": 0, "OTHER": 1},
    2: {"NSR": 0, "I-AVB": 2, "AF": 3, "OTHER": 1},
    3: {"NSR": 0, "I-AVB": 2, "AF": 3, "LBBB": 4, "RBBB": 5, "OTHER": 1},
    4: {"NSR": 0, "I-AVB": 2, "AF": 3, "LBBB": 4, "RBBB": 5, "PAC": 6, "PVC": 7, "STD": 8, "STE": 9}
}

def print_class_distribution(y, label_map):
    y = np.array(y).flatten()
    total = len(y)
    all_labels = sorted(label_map.values())
    print("\n📊 Class Distribution")
    for lbl in all_labels:
        count = np.sum(y == lbl)
        label = [k for k, v in label_map.items() if v == lbl]
        name = label[0] if label else str(lbl)
        print(f"  ├─ Label {lbl:<2} ({name:<10}) → {count:>5} samples ({(count/total)*100:5.2f}%)")

def ensure_folder(folder_path: str) -> None:
    """Ensure the given folder exists, create it if not."""
    os.makedirs(folder_path, exist_ok=True)


## 📦 EX. Load Example (Period 4) Data and View Format

This example demonstrates how to load preprocessed `.npy` data for **Period 4**, and inspect the dataset shapes and label distribution.  
Use this format as a reference when loading data in other methods (e.g., EWC, PNN, DynEx-CLoRA).

Each ECG sample:
- Has shape `(5000, 12)` → represents 10 seconds (at 500Hz) across 12-lead channels.
- Corresponding label is an integer ID (e.g., 0–9) defined by `period_label_map[4]`.

In [5]:
# # 範例:載入 period 4
# save_dir = os.path.join(BASE_DIR, "processed")
# X_train = np.load(os.path.join(save_dir, "X_train_p4.npy"))
# y_train = np.load(os.path.join(save_dir, "y_train_p4.npy"))
# X_test = np.load(os.path.join(save_dir, "X_test_p4.npy"))
# y_test = np.load(os.path.join(save_dir, "y_test_p4.npy"))

# print("✅ Loaded")
# print("X_train shape:", X_train.shape)
# print("y_train shape:", y_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_test shape:", y_test.shape)
# print_class_distribution(y_train, period_label_map[4])
# print_class_distribution(y_test, period_label_map[4])

# del X_train, y_train, X_test, y_test


## __Check GPU, CUDA, Pytorch__

In [6]:
!nvidia-smi

Sat May 10 15:40:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:2A:00.0 Off |                  Off |
| 30%   49C    P2             80W /  300W |    2713MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### CUDA Details

In [7]:
def check_gpu_config():
    """
    Check GPU availability and display detailed configuration information.
    """
    # Check if GPU is available
    gpu_available = torch.cuda.is_available()
    
    # Print header
    print("=" * 50)
    print("GPU Configuration Check".center(50))
    print("=" * 50)
    
    # Basic GPU availability
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'GPU Available':<25}: {'Yes' if gpu_available else 'No'}")
    
    # If GPU is available, print detailed info
    if gpu_available:
        print("-" * 50)
        print("GPU Details".center(50))
        print("-" * 50)
        
        # Device info
        print(f"{'Device Name':<25}: {torch.cuda.get_device_name(0)}")
        print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")
        print(f"{'Current Device Index':<25}: {torch.cuda.current_device()}")
        
        # Compute capability and CUDA cores
        props = torch.cuda.get_device_properties(0)
        print(f"{'Compute Capability':<25}: {props.major}.{props.minor}")
        print(f"{'Total CUDA Cores':<25}: {props.multi_processor_count * 128}")  # Approx. 128 cores per SM
        
        # Memory info
        total_memory = props.total_memory / (1024 ** 3)  # Convert to GB
        memory_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        memory_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        print(f"{'Total Memory (GB)':<25}: {total_memory:.2f}")
        print(f"{'Allocated Memory (GB)':<25}: {memory_allocated:.2f}")
        print(f"{'Reserved Memory (GB)':<25}: {memory_reserved:.2f}")
    else:
        print("-" * 50)
        print("No GPU detected. Running on CPU.".center(50))
        print("-" * 50)
    
    print("=" * 50)

if __name__ == "__main__":
    check_gpu_config()

             GPU Configuration Check              
PyTorch Version          : 2.5.1
GPU Available            : Yes
--------------------------------------------------
                   GPU Details                    
--------------------------------------------------
Device Name              : NVIDIA RTX A6000
Number of GPUs           : 3
Current Device Index     : 0
Compute Capability       : 8.6
Total CUDA Cores         : 10752
Total Memory (GB)        : 47.41
Allocated Memory (GB)    : 0.00
Reserved Memory (GB)     : 0.00


### PyTorch Details

In [8]:
def print_torch_config():
    """Print PyTorch and CUDA configuration in a formatted manner."""
    print("=" * 50)
    print("PyTorch Configuration".center(50))
    print("=" * 50)
    
    # Basic PyTorch and CUDA info
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'CUDA Compiled Version':<25}: {torch.version.cuda}")
    print(f"{'CUDA Available':<25}: {'Yes' if torch.cuda.is_available() else 'No'}")
    print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")

    # GPU details if available
    if torch.cuda.is_available():
        print(f"{'GPU Name':<25}: {torch.cuda.get_device_name(0)}")

    print("-" * 50)
    
    # Seed setting
    torch.manual_seed(42)
    print(f"{'Random Seed':<25}: 42 (Seeding successful!)")
    
    print("=" * 50)

if __name__ == "__main__":
    print_torch_config()

              PyTorch Configuration               
PyTorch Version          : 2.5.1
CUDA Compiled Version    : 12.1
CUDA Available           : Yes
Number of GPUs           : 3
GPU Name                 : NVIDIA RTX A6000
--------------------------------------------------
Random Seed              : 42 (Seeding successful!)


## __⚙️ GPU Selection — Auto-select the least loaded GPU__
This code automatically scans available GPUs and selects the one with the lowest current memory usage.


In [9]:
def auto_select_cuda_device(verbose=True):
    """
    Automatically selects the CUDA GPU with the least memory usage.
    Falls back to CPU if no GPU is available.
    """
    if not torch.cuda.is_available():
        print("🚫 No CUDA GPU available. Using CPU.")
        return torch.device("cpu")

    try:
        # Run nvidia-smi to get memory usage of each GPU
        smi_output = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        memory_used = [int(x) for x in smi_output.strip().split('\n')]
        best_gpu = int(np.argmin(memory_used))

        if verbose:
            print("🎯 Automatically selected GPU:")
            print(f"    - CUDA Device ID : {best_gpu}")
            print(f"    - Memory Used    : {memory_used[best_gpu]} MiB")
            print(f"    - Device Name    : {torch.cuda.get_device_name(best_gpu)}")
        return torch.device(f"cuda:{best_gpu}")
    except Exception as e:
        print(f"⚠️ Failed to auto-detect GPU. Falling back to cuda:0. ({e})")
        return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Execute and assign
device = auto_select_cuda_device()

🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 2713 MiB
    - Device Name    : NVIDIA RTX A6000


## __Model__

### ResNet 18 - 1D (ResNet18_1D_big_inplane)

In [ ]:
class BasicBlock1d(nn.Module):
    expansion = 1
    
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock1d, self).__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out

class ResNet18_1D(nn.Module):
    def __init__(self, input_channels=12, output_size=9, inplanes=64):
        super(ResNet18_1D, self).__init__()
        self.inplanes = inplanes
        
        # Initial conv layer
        self.conv1 = nn.Conv1d(input_channels, self.inplanes, kernel_size=15, stride=2, padding=7, bias=False)
        self.bn1 = nn.BatchNorm1d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers
        self.layer1 = self._make_layer(BasicBlock1d, 64, 2)
        self.layer2 = self._make_layer(BasicBlock1d, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock1d, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock1d, 512, 2, stride=2)
        
        # Adaptive pooling (both avg and max)
        self.adaptiveavgpool = nn.AdaptiveAvgPool1d(1)
        self.adaptivemaxpool = nn.AdaptiveMaxPool1d(1)
        
        # Fully connected layer with dropout
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(512 * BasicBlock1d.expansion * 2, output_size)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )
        
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Expect input shape: (batch_size, time_steps, channels)
        x = x.permute(0, 2, 1)  # → (batch_size, channels, time_steps)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # Apply both avg and max pooling
        x1 = self.adaptiveavgpool(x)
        x2 = self.adaptivemaxpool(x)
        
        # Concatenate pooling results
        x = torch.cat((x1, x2), dim=1)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Apply dropout
        x = self.dropout(x)
        
        # Final classification
        x = self.fc(x)
        
        return x
    

    def forward_features(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x1 = self.adaptiveavgpool(x)
        x2 = self.adaptivemaxpool(x)
        x = torch.cat((x1, x2), dim=1)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return x

### PNN Module

In [ ]:
class ECG_PNN_Column(nn.Module):
    def __init__(self, input_channels: int, output_dim: int):
        super(ECG_PNN_Column, self).__init__()
        self.feature_extractor = ResNet18_1D(input_channels=input_channels, output_size=0)  # 不用 output
        self.lateral_adapter = nn.Linear(1024, 1024)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(1024, output_dim)

    def forward(self, x: torch.Tensor, lateral_features: torch.Tensor) -> torch.Tensor:
        new_features = self.feature_extractor.forward_features(x)  # (B, 1024)
        fused = new_features + self.lateral_adapter(lateral_features)
        fused = self.relu(fused)
        fused = self.dropout(fused)
        return self.classifier(fused)

class ECG_ProgressiveNN(nn.Module):
    def __init__(self, base_model: nn.Module, new_column: ECG_PNN_Column):
        super(ECG_ProgressiveNN, self).__init__()
        self.base_model = base_model
        self.new_column = new_column
        for param in self.base_model.parameters():
            param.requires_grad = False

    def extract_base_features(self, x: torch.Tensor) -> torch.Tensor:
        if isinstance(self.base_model, ECG_ProgressiveNN):
            return self.base_model.forward_features(x)
        else:
            return self.base_model.forward_features(x)

    def get_base_logits(self, x: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            if isinstance(self.base_model, ECG_ProgressiveNN):
                return self.base_model(x)
            else:
                return self.base_model(x)

    def forward_features(self, x: torch.Tensor) -> torch.Tensor:
        return self.extract_base_features(x)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        base_features = self.extract_base_features(x)
        base_logits = self.get_base_logits(x)
        new_logits = self.new_column(x, lateral_features=base_features)
        return torch.cat([base_logits, new_logits], dim=-1)


In [ ]:
# class ECG_PNN_Column(nn.Module):
#     def __init__(self, input_dim: int, output_dim: int, hidden_dim: int = 512, dropout: float = 0.2):
#         super(ECG_PNN_Column, self).__init__()
#         self.adapter = nn.Linear(input_dim, input_dim)  # 讓輸出仍是 1024
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(dropout)
#         self.classifier = nn.Linear(input_dim, output_dim)
#         self.init_weights()

#     def init_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Linear):
#                 nn.init.xavier_uniform_(m.weight)
#                 nn.init.constant_(m.bias, 0)

#     def forward(self, new_features: torch.Tensor, lateral_features: torch.Tensor) -> torch.Tensor:
#         combined = new_features + self.adapter(lateral_features)  # [B, 1024]
#         combined = self.relu(combined)
#         combined = self.dropout(combined)
#         return self.classifier(combined)  # [B, output_dim]

# class ECG_ProgressiveNN(nn.Module):
#     def __init__(self, base_model: nn.Module, new_column: ECG_PNN_Column):
#         super(ECG_ProgressiveNN, self).__init__()
#         self.base_model = base_model
#         self.new_column = new_column

#         for param in self.base_model.parameters():
#             param.requires_grad = False

#     def forward_features(self, x: torch.Tensor) -> torch.Tensor:
#         with torch.no_grad():
#             if hasattr(self.base_model, "forward_features"):
#                 return self.base_model.forward_features(x)
#             else:
#                 # 手動還原 feature 提取流程（原 ResNet18_1D）
#                 x = x.permute(0, 2, 1)  # (B, 12, 5000)
#                 x = self.base_model.conv1(x)
#                 x = self.base_model.bn1(x)
#                 x = self.base_model.relu(x)
#                 x = self.base_model.maxpool(x)

#                 x = self.base_model.layer1(x)
#                 x = self.base_model.layer2(x)
#                 x = self.base_model.layer3(x)
#                 x = self.base_model.layer4(x)

#                 x1 = self.base_model.adaptiveavgpool(x)
#                 x2 = self.base_model.adaptivemaxpool(x)
#                 x = torch.cat((x1, x2), dim=1)
#                 x = x.view(x.size(0), -1)
#                 return x

#     def get_logits(self, base_features: torch.Tensor) -> torch.Tensor:
#         with torch.no_grad():
#             if isinstance(self.base_model, ECG_ProgressiveNN):
#                 return self.base_model.forward_from_features(base_features)
#             else:
#                 return self.base_model.fc(base_features)

#     def forward_from_features(self, base_features: torch.Tensor) -> torch.Tensor:
#         base_logits = self.get_logits(base_features)
#         new_logits = self.new_column(base_features, lateral_features=base_features)
#         return torch.cat([base_logits, new_logits], dim=-1)

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         base_features = self.forward_features(x)
#         return self.forward_from_features(base_features)


## __Training and validation function__

### Extra Function

In [12]:
def compute_classwise_accuracy(student_logits_flat, y_batch, class_correct, class_total):
    """
    Computes per-class accuracy by accumulating correct and total samples for each class using vectorized operations.
    
    Args:
        student_logits_flat (torch.Tensor): Model predictions (logits) in shape [batch_size * seq_len, output_size]
        y_batch (torch.Tensor): True labels in shape [batch_size * seq_len]
        class_correct (dict): Dictionary to store correct predictions per class
        class_total (dict): Dictionary to store total samples per class
    """
    # Ensure inputs are on the same device
    if student_logits_flat.device != y_batch.device:
        raise ValueError("student_logits_flat and y_batch must be on the same device")

    # Convert logits to predicted class indices
    predictions = torch.argmax(student_logits_flat, dim=-1)  # Shape: [batch_size * seq_len]

    # Compute correct predictions mask
    correct_mask = (predictions == y_batch)  # Shape: [batch_size * seq_len], boolean

    # Get unique labels in this batch
    unique_labels = torch.unique(y_batch)

    # Update class_total and class_correct using vectorized operations
    for label in unique_labels:
        label = label.item()  # Convert tensor to scalar
        if label not in class_total:
            class_total[label] = 0
            class_correct[label] = 0
        
        # Count total samples for this label
        label_mask = (y_batch == label)
        class_total[label] += label_mask.sum().item()
        
        # Count correct predictions for this label
        class_correct[label] += (label_mask & correct_mask).sum().item()

In [13]:
def get_model_parameter_info(model):
    total_params = sum(p.numel() for p in model.parameters())
    param_size_bytes = total_params * 4
    param_size_MB = param_size_bytes / (1024**2)
    return total_params, param_size_MB

In [14]:
def compute_class_weights(y: np.ndarray, num_classes: int, exclude_classes: list = None) -> torch.Tensor:
    """
    計算 class weights（inverse frequency）避免 class imbalance。
    可排除某些類別（如不存在的類別），這些類別的權重將設為 0。
    """
    exclude_classes = set(exclude_classes or [])
    class_sample_counts = np.bincount(y, minlength=num_classes)
    total_samples = len(y)

    weights = np.zeros(num_classes, dtype=np.float32)

    for cls in range(num_classes):
        if cls in exclude_classes:
            weights[cls] = 0.0
        else:
            count = class_sample_counts[cls]
            weights[cls] = total_samples / (count + 1e-6)

    # Normalize only non-excluded weights
    valid_mask = np.array([cls not in exclude_classes for cls in range(num_classes)])
    norm_sum = weights[valid_mask].sum()
    if norm_sum > 0:
        weights[valid_mask] /= norm_sum

    print("\n📊 Class Weights (normalized):")
    for i, w in enumerate(weights):
        status = " (excluded)" if i in exclude_classes else ""
        print(f"  - Class {i}: {w:.4f}{status}")
    
    return torch.tensor(weights, dtype=torch.float32)

### Training Function

In [15]:
def train_with_pnn_ecg(model, output_size, criterion, optimizer,
                       X_train, y_train, X_val, y_val,
                       scheduler=None, num_epochs=10, batch_size=64,
                       model_saving_folder=None, model_name=None,
                       stop_signal_file=None, period=None,
                       device=None):

    print("\n🚀 'train_with_pnn_ecg' started.")
    start_time = time.time()

    device = device or auto_select_cuda_device()
    model_name = model_name or 'model'
    model_saving_folder = model_saving_folder or './saved_models'

    if os.path.exists(model_saving_folder):
        shutil.rmtree(model_saving_folder)
        print(f"✅ Removed existing folder: {model_saving_folder}")
    os.makedirs(model_saving_folder, exist_ok=True)

    model.to(device)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.long).to(device)

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

    print("\n✅ Data Overview:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

    best_results = []

    for epoch in range(num_epochs):
        if stop_signal_file and os.path.exists(stop_signal_file):
            print("\n🛑 Stop signal detected. Exiting training loop.")
            break

        model.train()
        epoch_loss = 0.0
        class_correct, class_total = {}, {}

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * X_batch.size(0)
            compute_classwise_accuracy(outputs, y_batch, class_correct, class_total)

        train_loss = epoch_loss / len(train_loader.dataset)
        train_acc = {int(c): f"{(class_correct[c] / class_total[c]) * 100:.2f}%" if class_total[c] > 0 else "0.00%" for c in sorted(class_total.keys())}

        # ========== Validation ==========
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_class_correct, val_class_total = {}, {}

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                val_loss += criterion(outputs, y_batch).item() * X_batch.size(0)
                predictions = torch.argmax(outputs, dim=-1)
                val_correct += (predictions == y_batch).sum().item()
                val_total += y_batch.size(0)
                compute_classwise_accuracy(outputs, y_batch, val_class_correct, val_class_total)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / val_total
        val_acc_cls = {int(c): f"{(val_class_correct[c] / val_class_total[c]) * 100:.2f}%" if val_class_total[c] > 0 else "0.00%" for c in sorted(val_class_total.keys())}

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Train-Class-Acc: {train_acc}")
        print(f"Val Loss: {val_loss:.6f}, Val Acc: {val_acc * 100:.2f}%, Val-Class-Acc: {val_acc_cls}, LR: {optimizer.param_groups[0]['lr']:.6f}")

        # ========== Save Model ==========
        model_path = os.path.join(model_saving_folder, f"{model_name}_epoch_{epoch+1}.pth")
        current = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'train_classwise_accuracy': train_acc,
            'val_classwise_accuracy': val_acc_cls,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'learning_rate': optimizer.param_groups[0]['lr'],
            'model_path': model_path
        }

        if len(best_results) < 5 or val_acc > best_results[-1]['val_accuracy']:
            if len(best_results) == 5:
                to_remove = best_results.pop()
                if os.path.exists(to_remove['model_path']):
                    os.remove(to_remove['model_path'])
                    print(f"🗑 Removed: {to_remove['model_path']}")
            best_results.append(current)
            best_results.sort(key=lambda x: (x['val_accuracy'], x['epoch']), reverse=True)
            torch.save(current, model_path)
            print(f"✅ Saved model: {model_path}")

        if scheduler:
            scheduler.step(val_loss)

    # ========== Summary ==========
    training_time = time.time() - start_time
    total_params, param_size_MB = get_model_parameter_info(model)

    if best_results:
        best = best_results[0]
        best_model_path = os.path.join(model_saving_folder, f"{model_name}_best.pth")
        torch.save(best, best_model_path)
        print(f"\n🏆 Best model saved as: {best_model_path} (Val Accuracy: {best['val_accuracy'] * 100:.2f}%)")

    final_model_path = os.path.join(model_saving_folder, f"{model_name}_final.pth")
    torch.save(current, final_model_path)
    print(f"\n📌 Final model saved as: {final_model_path}")

    print("\n🎯 Top 5 Best Models:")
    for res in best_results:
        print(f"Epoch {res['epoch']}, Train Loss: {res['train_loss']:.6f}, Train-Acc: {res['train_classwise_accuracy']},\n"
              f"Val Loss: {res['val_loss']:.6f}, Val Acc: {res['val_accuracy']*100:.2f}%, Val-Acc: {res['val_classwise_accuracy']},"
              f" Model Path: {res['model_path']}")

    period_label = period or "?"
    model_name_str = model.__class__.__name__

    print(f"""
---
### Period {period_label}
+ ##### Total training time: {training_time:.2f} seconds
+ ##### Model: {model_name_str}
+ ##### Training and saving in *'{model_saving_folder}'*
+ ##### Best Epoch: {best['epoch']}
#### __Val Accuracy: {best['val_accuracy'] * 100:.2f}%__
#### __Val-Class-Acc: {best['val_classwise_accuracy']}__
#### __Total Parameters: {total_params:,}__
#### __Model Size (float32): {param_size_MB:.2f} MB__
""".strip())

    del X_train, y_train, X_val, y_val, train_loader, val_loader, current, outputs, predictions
    torch.cuda.empty_cache()
    gc.collect()


## __Training__

### Period 1 Summary
+ **Epoch:** 63
+ **Train Loss:** 0.0076635455248283595
+ **Val Loss:** 0.800983331773592
+ **Val Accuracy:** 88.86%
+ **Learning Rate:** 0.0006561000000000001
+ **Stored Model Path:** `Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v1/ResNet18_big_inplane_1D_epoch_63.pth`
+ **Train-Class-Acc:** {0: '99.86%', 1: '99.59%'}
+ **Val-Class-Acc:** {0: '91.85%', 1: '85.87%'}

In [16]:
def display_model_summary_with_params(model_folder, model_filename="ResNet18_big_inplane_1D_best.pth", input_channels=12, output_size=10):
    model_path = os.path.join(model_folder, model_filename)

    if not os.path.exists(model_path):
        print(f"❌ File not found: {model_path}")
        return

    checkpoint = torch.load(model_path, map_location='cpu')

    # === 還原模型並載入參數 ===
    model = ResNet18_1D(input_channels=input_channels, output_size=output_size)
    model.load_state_dict(checkpoint["model_state_dict"])
    total_params, param_size_MB = get_model_parameter_info(model)

    # === 顯示摘要 ===
    epoch = checkpoint.get("epoch", "?")
    train_loss = checkpoint.get("train_loss", "?")
    val_loss = checkpoint.get("val_loss", "?")
    val_acc = checkpoint.get("val_accuracy", "?")
    train_acc_dict = checkpoint.get("train_classwise_accuracy", {})
    val_acc_dict = checkpoint.get("val_classwise_accuracy", {})
    lr = checkpoint.get("learning_rate", "?")
    stored_path = checkpoint.get("model_path", "N/A")

    print(f"\n📦 Model Summary from: {model_path}")
    print(f"📌 Epoch: {epoch}")
    print(f"🧮 Train Loss: {train_loss:.6f}" if isinstance(train_loss, float) else f"🧮 Train Loss: {train_loss}")
    print(f"🎯 Val Loss: {val_loss:.6f}" if isinstance(val_loss, float) else f"🎯 Val Loss: {val_loss}")
    print(f"✅ Val Accuracy: {val_acc*100:.2f}%" if isinstance(val_acc, float) else f"✅ Val Accuracy: {val_acc}")
    print(f"📎 Learning Rate: {lr}")
    print(f"📁 Stored Model Path: {stored_path}")
    print(f"🧠 Total Parameters: {total_params:,}")
    print(f"📏 Model Size (float32): {param_size_MB:.2f} MB")

    print("\n📊 Train Class-wise Accuracy:")
    for c, acc in train_acc_dict.items():
        print(f"  └─ Class {c:<2}: {acc}")

    print("\n📊 Val Class-wise Accuracy:")
    for c, acc in val_acc_dict.items():
        print(f"  └─ Class {c:<2}: {acc}")

    print("\n---\n### Period 1 Summary (Markdown Format)")
    print(f"+ **Epoch:** {epoch}")
    print(f"+ **Train Loss:** {train_loss}")
    print(f"+ **Val Loss:** {val_loss}")
    print(f"+ **Val Accuracy:** {val_acc*100:.2f}%" if isinstance(val_acc, float) else f"+ **Val Accuracy:** {val_acc}")
    print(f"+ **Learning Rate:** {lr}")
    print(f"+ **Stored Model Path:** `{stored_path}`")
    print(f"+ **Total Parameters:** {total_params:,}")
    print(f"+ **Model Size (float32):** {param_size_MB:.2f} MB")
    print(f"+ **Train-Class-Acc:** {train_acc_dict}")
    print(f"+ **Val-Class-Acc:** {val_acc_dict}")
    print("---")

# Example call:
display_model_summary_with_params(
    model_folder=os.path.join("Class_Incremental_CL", "CPSC_CIL", "ResNet18_Selection", "ResNet18_big_inplane_v1"),
    input_channels=12,
    output_size=2
)


📦 Model Summary from: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v1/ResNet18_big_inplane_1D_best.pth
📌 Epoch: 63
🧮 Train Loss: 0.007664
🎯 Val Loss: 0.800983
✅ Val Accuracy: 88.86%
📎 Learning Rate: 0.0006561000000000001
📁 Stored Model Path: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v1/ResNet18_big_inplane_1D_epoch_63.pth
🧠 Total Parameters: 3,857,026
📏 Model Size (float32): 14.71 MB

📊 Train Class-wise Accuracy:
  └─ Class 0 : 99.86%
  └─ Class 1 : 99.59%

📊 Val Class-wise Accuracy:
  └─ Class 0 : 91.85%
  └─ Class 1 : 85.87%

---
### Period 1 Summary (Markdown Format)
+ **Epoch:** 63
+ **Train Loss:** 0.0076635455248283595
+ **Val Loss:** 0.800983331773592
+ **Val Accuracy:** 88.86%
+ **Learning Rate:** 0.0006561000000000001
+ **Stored Model Path:** `Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v1/ResNet18_big_inplane_1D_epoch_63.pth`
+ **Total Parameters:** 3,857,026
+ **Model Size (float32):** 14.71 MB
+

/tmp/ipykernel_588332/3192502623.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location='cpu')


### Period 2

In [25]:
# ================================
# 📌 Period 2: PNN Training
# ================================
period = 2

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v1", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 2 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels   = X_train.shape[2]               # 12-lead ECG
output_size      = len(np.unique(y_train))        # All known classes until now
new_output_size  = 2                               # Period 2 introduces 2 new classes
prev_output_size = output_size - new_output_size  # Period 1 had this many classes

# ==== Load Period 1 Best Model (含 FC 層) ====
prev_model_path = os.path.join(BASE_DIR, "ResNet18_Selection", "ResNet18_big_inplane_v1", "ResNet18_big_inplane_1D_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)

frozen_model = ResNet18_1D(input_channels=input_channels, output_size=prev_output_size)
frozen_model.load_state_dict(prev_checkpoint["model_state_dict"], strict=True)
frozen_model.to(device)
frozen_model.eval()
for param in frozen_model.parameters():
    param.requires_grad = False
print("✅ Loaded Period 1 frozen model (with FC)")

# ==== Create New Column ====
new_column = ECG_PNN_Column(input_dim=1024, output_dim=new_output_size, hidden_dim=512, dropout=0.2)

# ==== Wrap into PNN ====
model = ECG_ProgressiveNN(base_model=frozen_model, new_column=new_column).to(device)

# ==== Training Configuration ====
criterion      = nn.CrossEntropyLoss()
learning_rate  = 1e-3
weight_decay   = 1e-5
num_epochs     = 200
batch_size     = 64

optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
train_with_pnn_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_PNN",
    stop_signal_file=stop_signal_file,
    period=period,
    device=device
)

# ================================
# ✅ Cleanup
# ================================
del X_train, y_train, X_val, y_val
del frozen_model, new_column, model, prev_model_path, prev_checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 881 MiB
    - Device Name    : NVIDIA RTX A6000
✅ Loaded Period 1 frozen model (with FC)

🚀 'train_with_pnn_ecg' started.
✅ Removed existing folder: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v1/Period_2


/tmp/ipykernel_588332/3143211446.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)



✅ Data Overview:
X_train: torch.Size([3263, 5000, 12]), y_train: torch.Size([3263])
X_val: torch.Size([816, 5000, 12]), y_val: torch.Size([816])
Epoch 1/200, Train Loss: 9.728579, Train-Class-Acc: {0: '91.83%', 1: '39.75%', 2: '32.93%', 3: '42.83%'}
Val Loss: 2.103808, Val Acc: 62.25%, Val-Class-Acc: {0: '97.83%', 1: '25.00%', 2: '47.92%', 3: '81.15%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v1/Period_2/ResNet18_PNN_epoch_1.pth
Epoch 2/200, Train Loss: 2.210964, Train-Class-Acc: {0: '98.77%', 1: '40.37%', 2: '45.06%', 3: '56.86%'}
Val Loss: 2.031728, Val Acc: 54.53%, Val-Class-Acc: {0: '98.37%', 1: '65.16%', 2: '63.89%', 3: '5.33%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v1/Period_2/ResNet18_PNN_epoch_2.pth
Epoch 3/200, Train Loss: 1.876065, Train-Class-Acc: {0: '98.23%', 1: '46.52%', 2: '50.95%',

### Period 3

In [ ]:
# ================================
# 📌 Period 3: PNN Training
# ================================
period = 3

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v1", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 3 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels   = X_train.shape[2]
output_size      = len(np.unique(y_train))  # total classes up to now
new_output_size  = 2                        # this period introduces 2 new class

# ==== Reconstruct Period 2 PNN ====
base_model    = ResNet18_1D(input_channels=input_channels, output_size=2)
prev_column   = ECG_PNN_Column(input_dim=1024, output_dim=new_output_size, hidden_dim=512, dropout=0.2)
frozen_model  = ECG_ProgressiveNN(base_model=base_model, new_column=prev_column)

# ==== Load Period 2 Best Model State ====
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v1", "Period_2", "ResNet18_PNN_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(prev_checkpoint['model_state_dict'], strict=True)
frozen_model.to(device)
frozen_model.eval()
for param in frozen_model.parameters():
    param.requires_grad = False
print("✅ Loaded Period 2 frozen model (with FC + column)")

# ==== Create New Column ====
new_column = ECG_PNN_Column(input_dim=1024, output_dim=new_output_size, hidden_dim=512, dropout=0.2)

# ==== Wrap into New PNN ====
model = ECG_ProgressiveNN(base_model=frozen_model, new_column=new_column).to(device)

# ==== Training Configuration ====
criterion      = nn.CrossEntropyLoss()
learning_rate  = 1e-3
weight_decay   = 1e-5
num_epochs     = 200
batch_size     = 64

optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
train_with_pnn_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_PNN",
    stop_signal_file=stop_signal_file,
    period=period,
    device=device
)

# ==== Cleanup ====
del X_train, y_train, X_val, y_val
del base_model, prev_column, frozen_model, new_column, model, prev_model_path, prev_checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 883 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_588332/1112248827.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)


✅ Loaded Period 2 frozen model (with FC + column)

🚀 'train_with_pnn_ecg' started.
✅ Removed existing folder: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v1/Period_3

✅ Data Overview:
X_train: torch.Size([5120, 5000, 12]), y_train: torch.Size([5120])
X_val: torch.Size([1281, 5000, 12]), y_val: torch.Size([1281])
Epoch 1/200, Train Loss: 4.564051, Train-Class-Acc: {0: '92.92%', 1: '35.75%', 2: '89.08%', 3: '82.17%', 4: '15.82%', 5: '67.19%'}
Val Loss: 1.942662, Val Acc: 71.04%, Val-Class-Acc: {0: '97.28%', 1: '28.96%', 2: '88.19%', 3: '80.33%', 4: '25.00%', 5: '90.12%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v1/Period_3/ResNet18_PNN_epoch_1.pth
Epoch 2/200, Train Loss: 1.959776, Train-Class-Acc: {0: '96.19%', 1: '36.20%', 2: '91.51%', 3: '84.84%', 4: '35.44%', 5: '74.74%'}
Val Loss: 1.615601, Val Acc: 74.71%, Val-Class-Acc: {0: '97

In [ ]:
# ================================
# 📌 Period 3: PNN Training
# weight = class_weights_tensor
# ================================
period = 3

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v2", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 3 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels   = X_train.shape[2]
output_size      = len(np.unique(y_train))  # total classes up to now
new_output_size  = 2                        # this period introduces 2 new class

# ==== Reconstruct Period 2 PNN ====
base_model    = ResNet18_1D(input_channels=input_channels, output_size=2)
prev_column   = ECG_PNN_Column(input_dim=1024, output_dim=new_output_size, hidden_dim=512, dropout=0.2)
frozen_model  = ECG_ProgressiveNN(base_model=base_model, new_column=prev_column)

# ==== Load Period 2 Best Model State ====
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v1", "Period_2", "ResNet18_PNN_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(prev_checkpoint['model_state_dict'], strict=True)
frozen_model.to(device)
frozen_model.eval()
for param in frozen_model.parameters():
    param.requires_grad = False
print("✅ Loaded Period 2 frozen model (with FC + column)")

# ==== Create New Column ====
new_column = ECG_PNN_Column(input_dim=1024, output_dim=new_output_size, hidden_dim=512, dropout=0.2)

# ==== Wrap into New PNN ====
model = ECG_ProgressiveNN(base_model=frozen_model, new_column=new_column).to(device)

# ==== Training Configuration ====
class_weights_tensor = compute_class_weights(y_train, output_size)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))
learning_rate  = 1e-3
weight_decay   = 1e-5
num_epochs     = 200
batch_size     = 64

optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
train_with_pnn_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_PNN",
    stop_signal_file=stop_signal_file,
    period=period,
    device=device
)

# ==== Cleanup ====
del X_train, y_train, X_val, y_val
del base_model, prev_column, frozen_model, new_column, model, prev_model_path, prev_checkpoint
del class_weights_tensor
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 905 MiB
    - Device Name    : NVIDIA RTX A6000
✅ Loaded Period 2 frozen model (with FC + column)

📊 Class Weights (normalized):
  - Class 0: 0.1141
  - Class 1: 0.0626
  - Class 2: 0.1451
  - Class 3: 0.0858
  - Class 4: 0.5299
  - Class 5: 0.0626

🚀 'train_with_pnn_ecg' started.
✅ Removed existing folder: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v2/Period_3


/tmp/ipykernel_588332/846280196.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)



✅ Data Overview:
X_train: torch.Size([5120, 5000, 12]), y_train: torch.Size([5120])
X_val: torch.Size([1281, 5000, 12]), y_val: torch.Size([1281])
Epoch 1/200, Train Loss: 4.950151, Train-Class-Acc: {0: '91.55%', 1: '29.17%', 2: '85.44%', 3: '76.02%', 4: '50.63%', 5: '64.28%'}
Val Loss: 1.947719, Val Acc: 74.08%, Val-Class-Acc: {0: '96.74%', 1: '41.49%', 2: '89.58%', 3: '83.61%', 4: '27.50%', 5: '86.23%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v2/Period_3/ResNet18_PNN_epoch_1.pth
Epoch 2/200, Train Loss: 1.722893, Train-Class-Acc: {0: '97.14%', 1: '30.82%', 2: '88.73%', 3: '81.66%', 4: '66.46%', 5: '70.03%'}
Val Loss: 1.222663, Val Acc: 69.01%, Val-Class-Acc: {0: '97.28%', 1: '34.63%', 2: '88.89%', 3: '92.21%', 4: '87.50%', 5: '60.18%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v2/Period_3/ResNet18_

### Period 4

In [34]:
# ================================
# 📌 Period 4: PNN Training
# ================================
period = 4

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v1", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 4 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels   = X_train.shape[2]
output_size      = int(np.max(y_train)) + 1  # e.g., 6 → 10 classes total
new_output_size  = 4  # Period 4 新增四個類別（拆細 Walking）

# ==== Step 1: 還原 Period 2 的模型（Base + Column）====
base_model     = ResNet18_1D(input_channels=input_channels, output_size=2)  # Period 1 output
prev_column_2  = ECG_PNN_Column(input_dim=1024, output_dim=2, hidden_dim=512, dropout=0.2)
frozen_step1   = ECG_ProgressiveNN(base_model=base_model, new_column=prev_column_2)

# ==== Step 2: 接上 Period 3 的 Column ====
prev_column_3  = ECG_PNN_Column(input_dim=1024, output_dim=2, hidden_dim=512, dropout=0.2)
frozen_step2   = ECG_ProgressiveNN(base_model=frozen_step1, new_column=prev_column_3)

# ==== 載入 Period 3 完整模型 state_dict ====
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v1", "Period_3", "ResNet18_PNN_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)
frozen_step2.load_state_dict(prev_checkpoint["model_state_dict"], strict=True)
frozen_step2.to(device)
frozen_step2.eval()
for param in frozen_step2.parameters():
    param.requires_grad = False
print("✅ Loaded Period 3 frozen model (multi-column)")

# ==== Step 3: 加入 Period 4 的 Column ====
new_column_4 = ECG_PNN_Column(input_dim=1024, output_dim=new_output_size, hidden_dim=512, dropout=0.2)
model = ECG_ProgressiveNN(base_model=frozen_step2, new_column=new_column_4).to(device)

# ==== Training Configuration ====
criterion      = nn.CrossEntropyLoss()
learning_rate  = 1e-3
weight_decay   = 1e-5
num_epochs     = 200
batch_size     = 64

optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
train_with_pnn_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_PNN",
    stop_signal_file=stop_signal_file,
    period=period,
    device=device
)

# ==== Cleanup ====
del X_train, y_train, X_val, y_val
del base_model, prev_column_2, prev_column_3, new_column_4
del frozen_step1, frozen_step2, model, prev_model_path, prev_checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 911 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_588332/1394101560.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)


✅ Loaded Period 3 frozen model (multi-column)

🚀 'train_with_pnn_ecg' started.
✅ Removed existing folder: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v1/Period_4

✅ Data Overview:
X_train: torch.Size([5493, 5000, 12]), y_train: torch.Size([5493])
X_val: torch.Size([1374, 5000, 12]), y_val: torch.Size([1374])
Epoch 1/200, Train Loss: 4.490815, Train-Class-Acc: {0: '79.56%', 2: '76.43%', 3: '76.64%', 4: '82.91%', 5: '79.51%', 6: '16.36%', 7: '38.12%', 8: '45.54%', 9: '8.78%'}
Val Loss: 0.959420, Val Acc: 78.97%, Val-Class-Acc: {0: '93.48%', 2: '90.28%', 3: '93.03%', 4: '92.50%', 5: '91.04%', 6: '18.52%', 7: '79.20%', 8: '53.50%', 9: '29.73%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v1/Period_4/ResNet18_PNN_epoch_1.pth
Epoch 2/200, Train Loss: 1.193610, Train-Class-Acc: {0: '88.96%', 2: '81.11%', 3: '81.97%', 4: '89.87%', 5: '86.01%',

In [37]:
# ================================
# 📌 Period 4: PNN Training
# weight = class_weights_tensor
# ================================
period = 4

# ==== Paths ====
stop_signal_file = os.path.join(BASE_DIR, "stop_training.txt")
model_saving_folder = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v2", f"Period_{period}")
ensure_folder(model_saving_folder)

# ==== Load Period 4 Data ====
X_train = np.load(os.path.join(save_dir, f"X_train_p{period}.npy"))
y_train = np.load(os.path.join(save_dir, f"y_train_p{period}.npy"))
X_val   = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val   = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

# ==== Device ====
device = auto_select_cuda_device()

# ==== Model Configuration ====
input_channels   = X_train.shape[2]
output_size      = int(np.max(y_train)) + 1  # e.g., 6 → 10 classes total
new_output_size  = 4  # Period 4 新增四個類別（拆細 Walking）

# ==== Step 1: 還原 Period 2 的模型（Base + Column）====
base_model     = ResNet18_1D(input_channels=input_channels, output_size=2)  # Period 1 output
prev_column_2  = ECG_PNN_Column(input_dim=1024, output_dim=2, hidden_dim=512, dropout=0.2)
frozen_step1   = ECG_ProgressiveNN(base_model=base_model, new_column=prev_column_2)

# ==== Step 2: 接上 Period 3 的 Column ====
prev_column_3  = ECG_PNN_Column(input_dim=1024, output_dim=2, hidden_dim=512, dropout=0.2)
frozen_step2   = ECG_ProgressiveNN(base_model=frozen_step1, new_column=prev_column_3)

# ==== 載入 Period 3 完整模型 state_dict ====
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v2", "Period_3", "ResNet18_PNN_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)
frozen_step2.load_state_dict(prev_checkpoint["model_state_dict"], strict=True)
frozen_step2.to(device)
frozen_step2.eval()
for param in frozen_step2.parameters():
    param.requires_grad = False
print("✅ Loaded Period 3 frozen model (multi-column)")

# ==== Step 3: 加入 Period 4 的 Column ====
new_column_4 = ECG_PNN_Column(input_dim=1024, output_dim=new_output_size, hidden_dim=512, dropout=0.2)
model = ECG_ProgressiveNN(base_model=frozen_step2, new_column=new_column_4).to(device)

# ==== Training Configuration ====
exclude_labels = [1]  # 假設 y_train 沒有 class 1，但 output_size 還是包含它
class_weights_tensor = compute_class_weights(y_train, output_size, exclude_classes=exclude_labels)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))
learning_rate  = 1e-3
weight_decay   = 1e-5
num_epochs     = 200
batch_size     = 64

optimizer = torch.optim.Adam(model.new_column.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
train_with_pnn_ecg(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name="ResNet18_PNN",
    stop_signal_file=stop_signal_file,
    period=period,
    device=device
)

# ==== Cleanup ====
del X_train, y_train, X_val, y_val
del base_model, prev_column_2, prev_column_3, new_column_4
del frozen_step1, frozen_step2, model, prev_model_path, prev_checkpoint
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 913 MiB
    - Device Name    : NVIDIA RTX A6000
✅ Loaded Period 3 frozen model (multi-column)

📊 Class Weights (normalized):
  - Class 0: 0.0571
  - Class 1: 0.0000 (excluded)
  - Class 2: 0.0727
  - Class 3: 0.0430
  - Class 4: 0.2654
  - Class 5: 0.0314
  - Class 6: 0.0966
  - Class 7: 0.0837
  - Class 8: 0.0668
  - Class 9: 0.2833

🚀 'train_with_pnn_ecg' started.
✅ Removed existing folder: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v2/Period_4


/tmp/ipykernel_588332/799975350.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)



✅ Data Overview:
X_train: torch.Size([5493, 5000, 12]), y_train: torch.Size([5493])
X_val: torch.Size([1374, 5000, 12]), y_val: torch.Size([1374])
Epoch 1/200, Train Loss: 4.445167, Train-Class-Acc: {0: '68.12%', 2: '71.23%', 3: '70.08%', 4: '89.24%', 5: '67.46%', 6: '21.20%', 7: '44.71%', 8: '34.71%', 9: '27.03%'}
Val Loss: 1.353178, Val Acc: 65.14%, Val-Class-Acc: {0: '71.74%', 2: '73.61%', 3: '80.33%', 4: '95.00%', 5: '81.19%', 6: '53.70%', 7: '17.60%', 8: '28.66%', 9: '70.27%'}, LR: 0.001000
✅ Saved model: /mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL/Trained_models/PNN_CIL_v2/Period_4/ResNet18_PNN_epoch_1.pth
Epoch 2/200, Train Loss: 1.400788, Train-Class-Acc: {0: '81.06%', 2: '78.16%', 3: '72.95%', 4: '93.04%', 5: '75.17%', 6: '25.35%', 7: '55.49%', 8: '53.98%', 9: '36.49%'}
Val Loss: 0.974972, Val Acc: 78.68%, Val-Class-Acc: {0: '85.87%', 2: '87.50%', 3: '91.80%', 4: '95.00%', 5: '85.37%', 6: '5.56%', 7: '76.80%', 8: '80.89%', 9: '54.05%'}, 

##  Compute FWT $ Model Size

In [16]:
def compute_fwt_ecg(previous_model, init_model, X_val, y_val, known_classes, batch_size=64):
    """
    FWT computation for ECG-style inputs with 1D CNN (e.g., ResNet18_1D).
    X_val: shape [B, T, C]  (e.g., [N, 5000, 12])
    y_val: shape [B]        (e.g., [N])
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    previous_model.to(device).eval()
    init_model.to(device).eval()

    # 只選取 known classes
    mask = np.isin(y_val, known_classes)
    X_known = X_val[mask]
    y_known = y_val[mask]

    if len(y_known) == 0:
        print(f"⚠️ No validation samples for known classes {known_classes}.")
        return None, None, None

    print(f"📋 Total samples for known classes {known_classes}: {len(y_known)}")

    dataset = TensorDataset(
        torch.tensor(X_known, dtype=torch.float32),
        torch.tensor(y_known, dtype=torch.long)
    )
    loader = DataLoader(dataset, batch_size=batch_size)

    correct_prev, correct_init, total = 0, 0, 0

    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)

            out_prev = previous_model(xb)  # [B, C]
            out_init = init_model(xb)

            preds_prev = torch.argmax(out_prev, dim=-1)
            preds_init = torch.argmax(out_init, dim=-1)

            correct_prev += (preds_prev == yb).sum().item()
            correct_init += (preds_init == yb).sum().item()
            total += yb.size(0)

    acc_prev = 100 * correct_prev / total
    acc_init = 100 * correct_init / total
    fwt_value = acc_prev - acc_init

    print(f"\n### 🔍 FWT Debug Info:")
    print(f"- Total evaluated samples: {total}")
    print(f"- Accuracy by previous model: {acc_prev:.2f}%")
    print(f"- Accuracy by init model:     {acc_init:.2f}%")
    print(f"- FWT = Acc_prev - Acc_init = {fwt_value:.2f}%")

    return fwt_value, acc_prev, acc_init


### Period 2

In [ ]:
# ================================
# 📌 FWT - Period 2 (ECG PNN)
# ================================
period = 2

# === Load Validation Data ===
X_val = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

device = auto_select_cuda_device()
input_channels = X_val.shape[2]
output_size_prev = 2
known_classes = [0,1]

# === Load Period 1 frozen model ===
prev_model_path = os.path.join(BASE_DIR, "ResNet18_Selection", "ResNet18_big_inplane_v1", "ResNet18_big_inplane_1D_best.pth")
prev_checkpoint = torch.load(prev_model_path, map_location=device)

frozen_model = ResNet18_1D(input_channels=input_channels, output_size=output_size_prev).to(device)
frozen_model.load_state_dict(prev_checkpoint["model_state_dict"])
frozen_model.eval()
frozen_model.output_size = output_size_prev
del prev_checkpoint

# === Init Model ===
init_model = ResNet18_1D(input_channels=input_channels, output_size=output_size_prev).to(device)
init_model.output_size = output_size_prev

# === Tensor Conversion ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# === Run FWT Evaluation ===
fwt, acc_prev, acc_init = compute_fwt_ecg(frozen_model, init_model, X_val_tensor, y_val_tensor, known_classes)


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 2713 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_114062/988413461.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_model_path, map_location=device)


📋 Total samples for known classes [0, 1]: 428


/tmp/ipykernel_114062/1383618855.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(X_known, dtype=torch.float32),
/tmp/ipykernel_114062/1383618855.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_known, dtype=torch.long)



### 🔍 FWT Debug Info:
- Total evaluated samples: 428
- Accuracy by previous model: 89.25%
- Accuracy by init model:     42.99%
- FWT = Acc_prev - Acc_init = 46.26%


In [ ]:
# === 模擬輸入資料 ===
batch_size = 8
sequence_length = 5000
input_channels = 12
output_size = 4
new_output_size = 2
prev_output_size = output_size - new_output_size

dummy_input = torch.randn(batch_size, sequence_length, input_channels)  # [B, 5000, 12]

# === 建立 base model（period 1）===
base_model = ResNet18_1D(input_channels=input_channels, output_size=prev_output_size)

# === 建立完整模型並計算參數大小 ===
new_column = ECG_PNN_Column(input_channels=input_channels, output_dim=new_output_size)
model = ECG_ProgressiveNN(base_model=base_model, new_column=new_column)

total_params, param_size_MB = get_model_parameter_info(model)
print(f"\n📊 Simulated PNN Period 2 Model Size")
print(f"  - Total Parameters: {total_params:,}")
print(f"  - Model Size (float32): {param_size_MB:.2f} MB")



📊 Simulated PNN Period 2 Model Size
  - Total Parameters: 8,763,652
  - Model Size (float32): 33.43 MB


### Period 3

In [ ]:
# ================================
# 📌 FWT - Period 3 (ECG PNN)
# ================================
period = 3

# === Load Validation Data ===
X_val = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

device = auto_select_cuda_device()
input_channels = X_val.shape[2]
output_size_prev = 4
known_classes = [0, 1, 2, 3]

# === Step 1: Rebuild Period 2 PNN Model ===
base_model = ResNet18_1D(input_channels=input_channels, output_size=2)  # Period 1: 2 classes
prev_column = ECG_PNN_Column(input_dim=1024, output_dim=2, hidden_dim=512, dropout=0.2)
frozen_model = ECG_ProgressiveNN(base_model=base_model, new_column=prev_column).to(device)

# === Step 2: Load Period 2 state dict ===
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v1", "Period_2", "ResNet18_PNN_best.pth")
checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(checkpoint["model_state_dict"])
frozen_model.output_size = output_size_prev
frozen_model.eval()
del checkpoint

# === Init Model ===
init_model = ResNet18_1D(input_channels=input_channels, output_size=output_size_prev).to(device)
init_model.output_size = output_size_prev

# === Tensor Conversion ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# === Run FWT Evaluation ===
fwt, acc_prev, acc_init = compute_fwt_ecg(frozen_model, init_model, X_val_tensor, y_val_tensor, known_classes)


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 8282 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_114062/1887162379.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


📋 Total samples for known classes [0, 1, 2, 3]: 907


/tmp/ipykernel_114062/1383618855.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(X_known, dtype=torch.float32),
/tmp/ipykernel_114062/1383618855.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_known, dtype=torch.long)



### 🔍 FWT Debug Info:
- Total evaluated samples: 907
- Accuracy by previous model: 81.92%
- Accuracy by init model:     20.29%
- FWT = Acc_prev - Acc_init = 61.63%


In [ ]:
# ================================
# 📌 Simulated PNN Period 3 Size
# ================================
output_size = 6
new_output_size = 2
prev_output_size = output_size - new_output_size

# === 模擬輸入資料 ===
dummy_input = torch.randn(8, 5000, 12)

# === 建立 Period 1 base model ===
base_model = ResNet18_1D(input_channels=12, output_size=2)

# === Period 2 的 column（output=2）===
col2 = ECG_PNN_Column(input_channels=12, output_dim=2)
frozen_step1 = ECG_ProgressiveNN(base_model=base_model, new_column=col2)

# === Period 3 的 column（output=2）===
col3 = ECG_PNN_Column(input_channels=12, output_dim=new_output_size)
model = ECG_ProgressiveNN(base_model=frozen_step1, new_column=col3)

# === 計算參數大小 ===
total_params, param_size_MB = get_model_parameter_info(model)
print(f"\n📊 Simulated PNN Period 3 Model Size")
print(f"  - Total Parameters: {total_params:,}")
print(f"  - Model Size (float32): {param_size_MB:.2f} MB")



📊 Simulated PNN Period 3 Model Size
  - Total Parameters: 13,670,278
  - Model Size (float32): 52.15 MB


### Period 4

In [24]:
# ================================
# 📌 FWT - Period 4 (ECG PNN)
# ================================
period = 4

# === Load Validation Data ===
X_val = np.load(os.path.join(save_dir, f"X_test_p{period}.npy"))
y_val = np.load(os.path.join(save_dir, f"y_test_p{period}.npy"))

device = auto_select_cuda_device()
input_channels = X_val.shape[2]
output_size_prev = 6
known_classes = [0, 1, 2, 3, 4, 5]

# === Step 1: 還原 Period 2 結構 ===
base_model = ResNet18_1D(input_channels=input_channels, output_size=2)
prev_column_2 = ECG_PNN_Column(input_dim=1024, output_dim=2, hidden_dim=512, dropout=0.2)
frozen_step1 = ECG_ProgressiveNN(base_model, prev_column_2)

# === Step 2: 加上 Period 3 的 Column ===
prev_column_3 = ECG_PNN_Column(input_dim=1024, output_dim=2, hidden_dim=512, dropout=0.2)
frozen_model = ECG_ProgressiveNN(frozen_step1, prev_column_3).to(device)

# === 載入 Period 3 完整模型參數 ===
prev_model_path = os.path.join(BASE_DIR, "Trained_models", "PNN_CIL_v1", "Period_3", "ResNet18_PNN_best.pth")
checkpoint = torch.load(prev_model_path, map_location=device)
frozen_model.load_state_dict(checkpoint["model_state_dict"])
frozen_model.output_size = output_size_prev
frozen_model.eval()
del checkpoint

# === Init Model ===
init_model = ResNet18_1D(input_channels=input_channels, output_size=output_size_prev).to(device)
init_model.output_size = output_size_prev

# === Tensor Conversion ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# === Run FWT Evaluation ===
fwt, acc_prev, acc_init = compute_fwt_ecg(frozen_model, init_model, X_val_tensor, y_val_tensor, known_classes)


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 8312 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_114062/1918224278.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)


📋 Total samples for known classes [0, 1, 2, 3, 4, 5]: 947


/tmp/ipykernel_114062/1383618855.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(X_known, dtype=torch.float32),
/tmp/ipykernel_114062/1383618855.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_known, dtype=torch.long)



### 🔍 FWT Debug Info:
- Total evaluated samples: 947
- Accuracy by previous model: 96.09%
- Accuracy by init model:     25.45%
- FWT = Acc_prev - Acc_init = 70.64%


In [ ]:
# ================================
# 📌 Simulated PNN Period 4 Size
# ================================
output_size = 10
new_output_size = 4
prev_output_size = output_size - new_output_size

# === 模擬輸入資料 ===
dummy_input = torch.randn(8, 5000, 12)

# === Period 1 base model ===
base_model = ResNet18_1D(input_channels=12, output_size=2)

# === Period 2 column ===
col2 = ECG_PNN_Column(input_channels=12, output_dim=2)
frozen_step1 = ECG_ProgressiveNN(base_model=base_model, new_column=col2)

# === Period 3 column ===
col3 = ECG_PNN_Column(input_channels=12, output_dim=2)
frozen_step2 = ECG_ProgressiveNN(base_model=frozen_step1, new_column=col3)

# === Period 4 column ===
col4 = ECG_PNN_Column(input_channels=12, output_dim=new_output_size)
model = ECG_ProgressiveNN(base_model=frozen_step2, new_column=col4)

# === 計算參數大小 ===
total_params, param_size_MB = get_model_parameter_info(model)
print(f"\n📊 Simulated PNN Period 4 Model Size")
print(f"  - Total Parameters: {total_params:,}")
print(f"  - Model Size (float32): {param_size_MB:.2f} MB")



📊 Simulated PNN Period 4 Model Size
  - Total Parameters: 18,578,954
  - Model Size (float32): 70.87 MB


## 📊 Summary: 

### ✔️ CPSC - PNN: Validation Summary

| Period | Training Time (s) | Validation Accuracy | Class-wise Accuracy                                                                 |
|--------|-------------------|---------------------|--------------------------------------------------------------------------------------|
| 1      | 134.66            | **88.86%**          | {0: 91.85%, 1: 85.87%}                                                              |
| 2      | 120.70            | **81.13%**          | {0: 98.37%, 1: 63.93%, 2: 80.56%, 3: 85.66%}                                        |
| 3      | 191.86            | **80.25%**          | {0: 95.65%, 1: 51.64%, 2: 90.97%, 3: 89.75%, 4: 80.00%, 5: 88.92%}                  |
| 4      | 210.25            | **84.13%**          | {0: 88.04%, 2: 88.89%, 3: 92.62%, 4: 95.00%, 5: 92.24%, 6: 50.00%, 7: 78.40%, 8: 78.98%, 9: 45.95%} |


### 🧠 Continual Learning Metrics

| Period | AA_old (%) | AA_new (%) | BWT (%) | FWT (%) | FWT Classes        | Prev. Model Acc | Init Model Acc |
|--------|------------|------------|---------|---------|---------------------|------------------|-----------------|
| 2      | 81.15%     | 83.11%     | -7.71%  | 46.26%  | [0, 1]              | 89.25%           | 42.99%          |
| 3      | 82.00%     | 84.46%     | -0.13%  | 61.63%  | [0, 1, 2, 3]        | 81.92%           | 20.29%          |
| 4      | 91.36%     | 63.33%     | +2.30%  | 70.64%  | [0, 1, 2, 3, 4, 5]  | 96.09%           | 25.45%          |


### 📦 Model Size per Period

| Period | Output Size | Total Params | Δ Params vs Prev | Δ % vs Prev | Model Size (float32) |
|--------|-------------|--------------|------------------|-------------|-----------------------|
| 1      | 2           | 3,857,026    | —                | —           | 14.71 MB              |
| 2      | 4           | 8,763,652    | +4,906,626       | +127.2%     | 33.43 MB              |
| 3      | 6           |13,670,278    | +4,906,626       | +56.0%      | 52.15 MB              |
| 4      | 10          |18,578,954    | +4,908,676       | +35.9%      | 70.87 MB              |

**📈 Model Growth Rate (MGR) = (18,578,954 - 3,857,026) / (3,857,026 × 3) ≈ +127.1%**

**📈 Max trainable ratio = 4,906,626 / 8,763,652 ≈ 55.97%**
